In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
import xgboost
from gmdhpy import gmdh 
from sklearn.model_selection import RandomizedSearchCV
import pickle

from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer

In [2]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join('/home/osboxes/repos/sport/competitions_sendy/data/raw/SampleSubmission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='Order_No')
    
    submission['Time from Pickup to Arrival'] = prediction
    submission.to_csv(filename)

In [3]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [4]:
train = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/train_2.csv', sep=',') 

In [5]:
test = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/test_2.csv', sep=',') 

In [7]:
target = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/target_2.csv', sep=',') 

In [8]:
y_target = target['Time from Pickup to Arrival']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train[train_cols], y_target,
                                                    train_size=0.75, test_size=0.25)


In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15900, 27), (5301, 27), (15900,), (5301,))

In [14]:
# Make a custom a scorer from the custom metric function
# Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized.
my_custom_scorer = make_scorer(rmse, greater_is_better=False)

In [15]:
tpot = TPOTRegressor(generations=5, population_size=50, n_jobs=-1, cv=3, verbosity=2, scoring=my_custom_scorer, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('zindi_pipeline.py')


Imputing missing values in feature set


Generation 1 - Current best internal CV score: -760.5350414107107
Generation 2 - Current best internal CV score: -760.0897434462581
Generation 3 - Current best internal CV score: -757.1979679032725
Generation 4 - Current best internal CV score: -757.1979679032725
Generation 5 - Current best internal CV score: -757.1979679032725

Best pipeline: XGBRegressor(input_matrix, learning_rate=0.1, max_depth=5, min_child_weight=14, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.9000000000000001)
Imputing missing values in feature set
-753.6749305360953
